# Bayesian Temporal Matrix Factorization

**Published**: October 8, 2019

**Author**: Xinyu Chen [[**GitHub homepage**](https://github.com/xinychen)]

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the repository of [**tensor-learning**](https://github.com/xinychen/tensor-learning/blob/master/content/BTMF.ipynb).

## Abstract

Large-scale and multidimensional spatiotemporal data sets are becoming ubiquitous in many real-world applications such as monitoring traffic and air quality. Making predictions on these time series has become a critical challenge due to not only the large-scale and high-dimensional nature but also the considerable amount of missing data. In this work, we propose a Bayesian Temporal Matrix Factorization (BTMF) model for modeling multidimensional time series - and in particular spatiotemporal data - in the presence of missing data. By integrating low-rank matrix factorization and vector autoregressive (VAR) process into a single probabilistic graphical model, our model can effectively perform predictions without imputing those missing values. We develop efficient Gibbs sampling algorithms for model inference and test the proposed BTMF on several real-world spatiotemporal data sets for both missing data imputation and short-term rolling prediction tasks. This post is mainly about BTMF models and their **`Python`** implementation with an application of spatiotemporal data imputation.

## 1 Motivation



## 2 Problem Description

We assume a spatiotemporal setting for multidimensional time series data throughout this work. In general, modern spatiotemporal data sets collected from sensor networks can be organized as matrix time series. For example, we can denote by matrix $Y\in\mathbb{R}^{N\times T}$ a multivariate time series collected from $N$ locations/sensors on $T$ time stamps, with each row $$\boldsymbol{y}_{i}=\left(y_{i,1},y_{i,2},...,y_{i,t-1},y_{i,t},y_{i,t+1},...,y_{i,T}\right)$$
corresponding to the time series collected at location $i$.

As mentioned, making accurate predictions on incomplete time series is very challenging, while missing data problem is almost inevitable in real-world applications. Figure 1 illustrates the prediction problem for incomplete time series data. Here we use $(i,t)\in\Omega$ to index the observed entries in matrix $Y$.

<img src="../images/graphical_matrix_time_series.png" alt="drawing" width="500"/>

> **Figure 1**: Illustration of multivariate time series and the prediction problem in the presence of missing values (green: observed data; white: missing data; red: prediction).


## 3 Model Description

Given a partially observed spatiotemporal matrix $Y\in\mathbb{R}^{N \times T}$, one can factorize it into a spatial factor matrix $W\in\mathbb{R}^{R \times N}$ and a temporal factor matrix $X\in\mathbb{R}^{R \times T}$ following general matrix factorization model:
\begin{equation}
Y\approx W^{\top}X,
\label{btmf_equation1}
\end{equation}
and element-wise, we have
\begin{equation}
y_{it}\approx \boldsymbol{w}_{i}^\top\boldsymbol{x}_{t}, \quad \forall (i,t),
\label{btmf_equation2}
\end{equation}
where vectors $\boldsymbol{w}_{i}$ and $\boldsymbol{x}_{t}$ refer to the $i$-th column of $W$ and the $t$-th column of $X$, respectively.

The standard matrix factorization model is a good approach to deal with the missing data problem; however, it cannot capture the dependencies among different columns in $X$, which are critical in modeling time series data. To better characterize the temporal dependencies and impose temporal smoothness, a novel AR regularizer is introduced on $X$ in TRMF (i.e., Temporal Regularizer Matrix Factorization proposed by [Yu et al., 2016](https://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf)):
\begin{equation} \label{equ:VAR}
\begin{aligned}
    \boldsymbol{x}_{t+1}&=\sum\nolimits_{k=1}^{d}A_{k}\boldsymbol{x}_{t+1-h_k}+\boldsymbol{\epsilon}_t, \\
    &=A^\top \boldsymbol{v}_{t+1}+\boldsymbol{\epsilon}_{t}, \\
\end{aligned}
\end{equation}
where $\mathcal{L}=\left\{h_1,\ldots,h_k,\ldots,h_d\right\}$ is a lag set ($d$ is the order of this AR model), each $A_k$ ($k\in\left\{1,...,d\right\}$) is a $R\times R$ coefficient matrix, and $\boldsymbol{\epsilon}_t$ is a zero mean Gaussian noise vector. For brevity, matrix $A\in \mathbb{R}^{(R d) \times R}$ and vector $\boldsymbol{v}_{t+1}\in \mathbb{R}^{(R d) \times 1}$ are defined as
\begin{equation*}
A=\left[A_{1}, \ldots, A_{d}\right]^{\top} ,\quad \boldsymbol{v}_{t+1}=\left[\begin{array}{c}{\boldsymbol{x}_{t+1-h_1}} \\ {\vdots} \\ {\boldsymbol{x}_{t+1-h_d}}\end{array}\right] .
\end{equation*}

<img src="../images/rolling_prediction.png" alt="drawing" width="400"/>

> **Figure 2**: A graphical illustration of the rolling prediction scheme using BTMF (with VAR process) (green: observed data; white: missing data; red: prediction).

In [Yu et al., 2016](https://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf), to avoid overfitting and reduce the number of parameters, the coefficient matrix in TRMF is further assumed to be a diagonal $A_k=\text{diag}(\boldsymbol{\theta}_{k})$. Therefore, they have
\begin{equation} \label{equ:AR}
\boldsymbol{x}_{t+1}=\boldsymbol{\theta}_{1}\circledast\boldsymbol{x}_{t+1-h_1}+\cdots+\boldsymbol{\theta}_{d}\circledast\boldsymbol{x}_{t+1-h_d}+\boldsymbol{\epsilon}_t,
\end{equation}
where the symbol $\circledast$ denotes the element-wise Hadamard product. However, unlike Equation (4), a vector autoregressive (VAR) model in Equation (3) is actually more powerful for capturing multivariate time series patterns. 

<img src="../images/rolling_prediction_strategy.png" alt="drawing" width="400"/>

> **Figure 3**: A graphical illustration of the rolling prediction scheme using BTMF (with AR process) (green: observed data; white: missing data; red: prediction).

In the following, we first introduce a Bayesian temporal matrix factorization model with an autoregressive model given in Equation (4), and then discuss another model with a vector autoregressive (VAR) model shown in Equation (3).



## 4 Bayesian Sequential Matrix Factorization (BSMF)

## 5 Bayesian Temporal Matrix Factorization with Vector Autoregressive Model

### 5.1 Model Specification

Following the general Bayesian probabilistic matrix factorization models (e.g., BPMF proposed by [Salakhutdinov & Mnih, 2008](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf)), we assume that each observed entry in $Y$ follows a Gaussian distribution with precision $\tau$:
\begin{equation}
y_{i,t}\sim\mathcal{N}\left(\boldsymbol{w}_i^\top\boldsymbol{x}_t,\tau^{-1}\right),\quad \left(i,t\right)\in\Omega.
\label{btmf_equation3}
\end{equation}

On the spatial dimension, we use a simple Gaussian factor matrix without imposing any dependencies explicitly:
\begin{equation}
\boldsymbol{w}_i\sim\mathcal{N}\left(\boldsymbol{\mu}_{w},\Lambda_w^{-1}\right),
\end{equation}
and we place a conjugate Gaussian-Wishart prior on the mean vector and the precision matrix:
\begin{equation}
\boldsymbol{\mu}_w | \Lambda_w \sim\mathcal{N}\left(\boldsymbol{\mu}_0,(\beta_0\Lambda_w)^{-1}\right),\Lambda_w\sim\mathcal{W}\left(W_0,\nu_0\right),
\end{equation}
where $\boldsymbol{\mu}_0\in \mathbb{R}^{R}$ is a mean vector, $\mathcal{W}\left(W_0,\nu_0\right)$ is a Wishart distribution with a $R\times R$ scale matrix $W_0$ and $\nu_0$ degrees of freedom.

In modeling the temporal factor matrix $X$, we re-write the VAR process as:
\begin{equation}
\begin{aligned}
\boldsymbol{x}_{t}&\sim\begin{cases}
\mathcal{N}\left(\boldsymbol{0},I_R\right),&\text{if $t\in\left\{1,2,...,h_d\right\}$}, \\
\mathcal{N}\left(A^\top \boldsymbol{v}_{t},\Sigma\right),&\text{otherwise},\\
\end{cases}\\
\end{aligned}
\label{btmf_equation5}
\end{equation}

Since the mean vector is defined by VAR, we need to place the conjugate matrix normal inverse Wishart (MNIW) prior on the coefficient matrix $A$ and the covariance matrix $\Sigma$ as follows,
\begin{equation}
\begin{aligned}
A\sim\mathcal{MN}_{(Rd)\times R}\left(M_0,\Psi_0,\Sigma\right),\quad
\Sigma \sim\mathcal{IW}\left(S_0,\nu_0\right), \\
\end{aligned}
\end{equation}
where the probability density function for the $Rd$-by-$R$ random matrix $A$ has the form:
\begin{equation}
\begin{aligned}
&p\left(A\mid M_0,\Psi_0,\Sigma\right) \\
=&\left(2\pi\right)^{-R^2d/2}\left|\Psi_0\right|^{-R/2}\left|\Sigma\right|^{-Rd/2} \\
&\times \exp\left(-\frac{1}{2}\text{tr}\left[\Sigma^{-1}\left(A-M_0\right)^{\top}\Psi_{0}^{-1}\left(A-M_0\right)\right]\right), \\
\end{aligned}
\label{mnpdf}
\end{equation}
where $\Psi_0\in\mathbb{R}^{(Rd)\times (Rd)}$ and $\Sigma\in\mathbb{R}^{R\times R}$ are played as covariance matrices.

For the only remaining parameter $\tau$, we place a Gamma prior  $\tau\sim\text{Gamma}\left(\alpha,\beta\right)$ where $\alpha$ and $\beta$ are the shape and rate parameters, respectively. 

The above specifies the full generative process of BTMF, and we could also see the Bayesian graphical model shown in Figure 4. Several parameters are introduced to define the prior distributions for hyperparameters, including $\boldsymbol{\mu}_{0}$, $W_0$, $\nu_0$, $\beta_0$, $\alpha$, $\beta$, $M_0$, $\Psi_0$, and $S_0$. These parameters need to provided in advance when training the model. However, it should be noted that the specification of these parameters has little impact on the final results, as the training data will play a much more important role in defining the posteriors of the hyperparameters.

<img src="../images/btmf_net.png" alt="drawing" width="450"/>

> **Figure 4**: An overview graphical model of BTMF (time lag set: $\left\{1,2,...,d\right\}$). The shaded nodes ($y_{i,t}$) are the observed data in $\Omega$.


### 5.2 Model Inference



Given the complex structure of BTMF, it is intractable to write down the posterior distribution. Here we rely on the MCMC technique for Bayesian learning. In detail, we introduce a Gibbs sampling algorithm by deriving the full conditional distributions for all parameters and hyperparameters. Thanks to the use of conjugate priors in Figure 4, we can actually write down all the conditional distributions analytically. Below we summarize the Gibbs sampling procedure.  


#### 1) Sampling Factor Matrix $W$ and Its Hyperparameters



> For programming convenience, we use $W\in\mathbb{R}^{N\times R}$ to replace $W\in\mathbb{R}^{R\times N}$.

In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart

def cov_mat(mat):
    new_mat = mat - np.mean(mat, axis = 0)
    return np.einsum('ti, tj -> ij', new_mat, new_mat)

def sample_factor_w(sparse_mat, binary_mat, W, X, tau):
    """Sampling N-by-R factor matrix W and its hyperparameters (mu_w, Lambda_w)."""
    dim1, rank = W.shape
    beta0 = 1
    W_bar = np.mean(W, axis = 0)
    var_mu_hyper = (dim1 * W_bar) / (dim1 + beta0)
    var_W_hyper = inv(np.eye(rank) + cov_mat(W) + dim1 * beta0 / (dim1 + beta0) * np.outer(W_bar, W_bar))
    var_Lambda_hyper = wishart(df = dim1 + rank, scale = var_W_hyper, seed = None).rvs()
    var_mu_hyper = mvnrnd(var_mu_hyper, inv((dim1 + beta0) * var_Lambda_hyper))
    for i in range(dim1):
        pos0 = np.where(sparse_mat[i, :] != 0)
        Xt = X[pos0[0], :]
        var_mu = tau * np.matmul(Xt.T, sparse_mat[i, pos0[0]]) + np.matmul(var_Lambda_hyper, var_mu_hyper)
        inv_var_Lambda = inv(tau * np.matmul(Xt.T, Xt) + var_Lambda_hyper)
        W[i, :] = mvnrnd(np.matmul(inv_var_Lambda, var_mu), inv_var_Lambda)
    return W

#### 2) Sampling VAR Coefficients $A$ and Its Hyperparameters

**Foundations of VAR**

Vector autoregression (VAR) is a multivariate extension of autoregression (AR). Formally, VAR for $R$-dimensional vectors $\boldsymbol{x}_{t}$ can be written as follows,
\begin{equation}
\begin{aligned}
\boldsymbol{x}_{t}&=A_{1} \boldsymbol{x}_{t-h_1}+\cdots+A_{d} \boldsymbol{x}_{t-h_d}+\boldsymbol{\epsilon}_{t}, \\
&= A^\top \boldsymbol{v}_{t}+\boldsymbol{\epsilon}_{t},~t=h_d+1, \ldots, T, \\
\end{aligned}
\end{equation}
where
\begin{equation}
A=\left[A_{1}, \ldots, A_{d}\right]^{\top} \in \mathbb{R}^{(R d) \times R},\quad \boldsymbol{v}_{t}=\left[\begin{array}{c}{\boldsymbol{x}_{t-h_1}} \\ {\vdots} \\ {\boldsymbol{x}_{t-h_d}}\end{array}\right] \in \mathbb{R}^{(R d) \times 1}.
\end{equation}

In the following, if we define
\begin{equation}
Z=\left[\begin{array}{c}{\boldsymbol{x}_{h_d+1}^{\top}} \\ {\vdots} \\ {\boldsymbol{x}_{T}^{\top}}\end{array}\right] \in \mathbb{R}^{(T-h_d) \times R},\quad Q=\left[\begin{array}{c}{\boldsymbol{v}_{h_d+1}^{\top}} \\ {\vdots} \\ {\boldsymbol{v}_{T}^{\top}}\end{array}\right] \in \mathbb{R}^{(T-h_d) \times(R d)},
\end{equation}
then, we could write the above mentioned VAR as
\begin{equation}
\underbrace{Z}_{(T-h_d)\times R}\approx \underbrace{Q}_{(T-h_d)\times (Rd)}\times \underbrace{A}_{(Rd)\times R}.
\end{equation}

> To include temporal factors $\boldsymbol{x}_{t},t=1,...,h_d$, we also define $$Z_0=\left[\begin{array}{c}{\boldsymbol{x}_{1}^{\top}} \\ {\vdots} \\ {\boldsymbol{x}_{h_d}^{\top}}\end{array}\right] \in \mathbb{R}^{h_d \times R}.$$



**Build a Bayesian VAR on temporal factors $\boldsymbol{x}_{t}$**
\begin{equation}
\begin{aligned}
\boldsymbol{x}_{t}&\sim\begin{cases}\mathcal{N}\left(A^\top \boldsymbol{v}_{t},\Sigma\right),~\text{if $t\in\left\{h_d+1,...,T\right\}$},\\{\mathcal{N}\left(\boldsymbol{0},I_R\right),~\text{otherwise}}.\end{cases}\\
A&\sim\mathcal{MN}_{(Rd)\times R}\left(M_0,\Psi_0,\Sigma\right), \\
\Sigma &\sim\mathcal{IW}\left(S_0,\nu_0\right), \\
\end{aligned}
\end{equation}
where
\begin{equation}
\begin{aligned}
&\mathcal{M N}_{(R d) \times R}\left(A | M_{0}, \Psi_{0}, \Sigma\right)\\
\propto|&\Sigma|^{-R d / 2} \exp \left(-\frac{1}{2} \operatorname{tr}\left[\Sigma^{-1}\left(A-M_{0}\right)^{\top} \Psi_{0}^{-1}\left(A-M_{0}\right)\right]\right), \\
\end{aligned}
\end{equation}
and
\begin{equation}
\mathcal{I} \mathcal{W}\left(\Sigma | S_{0}, \nu_{0}\right) \propto|\Sigma|^{-\left(\nu_{0}+R+1\right) / 2} \exp \left(-\frac{1}{2} \operatorname{tr}\left(\Sigma^{-1}S_{0}\right)\right).
\end{equation}


**Likelihood from temporal factors $\boldsymbol{x}_{t}$**
\begin{equation}
\begin{aligned}
&\mathcal{L}\left(X\mid A,\Sigma\right) \\
\propto &\prod_{t=1}^{h_d}p\left(\boldsymbol{x}_{t}\mid \Sigma\right)\times \prod_{t=h_d+1}^{T}p\left(\boldsymbol{x}_{t}\mid A,\Sigma\right) \\
\propto &\left|\Sigma\right|^{-T/2}\exp\left\{-\frac{1}{2}\sum_{t=h_d+1}^{T}\left(\boldsymbol{x}_{t}-A^\top \boldsymbol{v}_{t}\right)^\top\Sigma^{-1}\left(\boldsymbol{x}_{t}-A^\top \boldsymbol{v}_{t}\right)\right\} \\
\propto &\left|\Sigma\right|^{-T/2}\exp\left\{-\frac{1}{2}\text{tr}\left[\Sigma^{-1}\left(Z_0^\top Z_0+\left(Z-QA\right)^\top \left(Z-QA\right)\right)\right]\right\}
\end{aligned}
\end{equation}


**Posterior distribution**

Consider
\begin{equation}
\begin{aligned}
&\left(A-M_{0}\right)^{\top} \Psi_{0}^{-1}\left(A-M_{0}\right)+S_0+Z_0^\top Z_0+\left(Z-QA\right)^\top \left(Z-QA\right) \\
=&A^\top\left(\Psi_0^{-1}+Q^\top Q\right)A-A^\top\left(\Psi_0^{-1}M_0+Q^\top Z\right) \\
&-\left(\Psi_0^{-1}M_0+Q^\top Z\right)^\top A \\
&+\left(\Psi_0^{-1}M_0+Q^\top Z\right)^\top\left(\Psi_0^{-1}+Q^\top Q\right)\left(\Psi_0^{-1}M_0+Q^\top Z\right) \\
&-\left(\Psi_0^{-1}M_0+Q^\top Z\right)^\top\left(\Psi_0^{-1}+Q^\top Q\right)\left(\Psi_0^{-1}M_0+Q^\top Z\right) \\
&+M_0^\top\Psi_0^{-1}M_0+S_0+Z_0^\top Z_0+Z^\top Z \\
=&\left(A-M^{*}\right)^\top\left(\Psi^{*}\right)^{-1}\left(A-M^{*}\right)+S^{*}, \\
\end{aligned}
\end{equation}
which is in the form of $\mathcal{MN}\left(\cdot\right)$ and $\mathcal{IW}\left(\cdot\right)$.

The $Rd$-by-$R$ matrix $A$ has a matrix normal distribution, and $R$-by-$R$ covariance matrix $\Sigma$ has an inverse Wishart distribution, that is,
\begin{equation}
A \sim \mathcal{M N}_{(R d) \times R}\left(M^{*}, \Psi^{*}, \Sigma\right), \quad \Sigma \sim \mathcal{I} \mathcal{W}\left(S^{*}, \nu^{*}\right),
\end{equation}
with
\begin{equation}
\begin{cases}
{\Psi^{*}=\left(\Psi_{0}^{-1}+Q^{\top} Q\right)^{-1}}, \\ {M^{*}=\Psi^{*}\left(\Psi_{0}^{-1} M_{0}+Q^{\top} Z\right)}, \\ {S^{*}=S_{0}+Z^\top Z+M_0^\top\Psi_0^{-1}M_0-\left(M^{*}\right)^\top\left(\Psi^{*}\right)^{-1}M^{*}}, \\ 
{\nu^{*}=\nu_{0}+T-h_d}.
\end{cases}
\end{equation}

In [2]:
from scipy.stats import invwishart

def mnrnd(M, U, V):
    """
    Generate matrix normal distributed random matrix.
    M is a m-by-n matrix, U is a m-by-m matrix, and V is a n-by-n matrix.
    """
    dim1, dim2 = M.shape
    X0 = np.random.rand(dim1, dim2)
    P = np.linalg.cholesky(U)
    Q = np.linalg.cholesky(V)
    return M + np.matmul(np.matmul(P, X0), Q.T)

def sample_var_coefficient(X, time_lags):
    dim2, rank = X.shape
    d = time_lags.shape[0]
    Z_mat = X[np.max(time_lags) : dim2, :]
    Q_mat = X[np.max(time_lags) - time_lags[0] : dim2 - time_lags[0], :]
    for k in range(1, d):
        Q_mat = np.append(Q_mat, X[np.max(time_lags) - time_lags[k] : dim2 - time_lags[k], :], axis = 1)
    var_Psi = inv(np.eye(rank * d) + np.matmul(Q_mat.T, Q_mat))
    var_M = np.matmul(var_Psi, np.matmul(Q_mat.T, Z_mat))
    var_S = (np.eye(rank) + np.matmul(Z_mat.T, Z_mat) - np.matmul(np.matmul(var_M.T, inv(var_Psi)), var_M))
    Sigma = invwishart(df = rank + dim2 - np.max(time_lags), scale = var_S, seed = None).rvs()
    return mnrnd(var_M, var_Psi, Sigma), Sigma

#### 3) Sampling Factor Matrix $X$

**Posterior distribution**
\begin{equation}
\begin{aligned}
y_{it}&\sim\mathcal{N}\left(\boldsymbol{w}_{i}^\top\boldsymbol{x}_{t},\tau^{-1}\right),~\left(i,t\right)\in\Omega, \\
\boldsymbol{x}_{t}&\sim\begin{cases}\mathcal{N}\left(\sum_{k=1}^{d}A_{k} \boldsymbol{x}_{t-h_k},\Sigma\right),~\text{if $t\in\left\{h_d+1,...,T\right\}$},\\{\mathcal{N}\left(\boldsymbol{0},I\right),~\text{otherwise}}.\end{cases}\\
\end{aligned}
\end{equation}

If $t\in\left\{1,...,h_d\right\}$, parameters of the posterior distribution $\mathcal{N}\left(\boldsymbol{x}_{t}\mid \boldsymbol{\mu}_{t}^{*},\Sigma_{t}^{*}\right)$ are
\footnotesize{
\begin{equation}
\begin{aligned}
\Sigma_{t}^{*}&=\left(\sum_{k=1, h_{d}<t+h_{k} \leq T}^{d} {A}_{k}^{\top} \Sigma^{-1} A_{k}+\tau\sum_{i:(i,t)\in\Omega}\boldsymbol{w}_{i}\boldsymbol{w}_{i}^\top+I\right)^{-1}, \\
\boldsymbol{\mu}_{t}^{*}&=\Sigma_{t}^{*}\left(\sum_{k=1, h_{d}<t+h_{k} \leq T}^{d} A_{k}^{\top} \Sigma^{-1} \boldsymbol{\psi}_{t+h_{k}}+\tau\sum_{i:(i,t)\in\Omega}\boldsymbol{w}_{i}y_{it}\right). \\
\end{aligned}
\end{equation}

If $t\in\left\{h_d+1,...,T\right\}$, then parameters of the posterior distribution $\mathcal{N}\left(\boldsymbol{x}_{t}\mid \boldsymbol{\mu}_{t}^{*},\Sigma_{t}^{*}\right)$ are
\begin{equation}
\begin{aligned}
\Sigma_{t}^{*}&=\left(\sum_{k=1, h_{d}<t+h_{k} \leq T}^{d} {A}_{k}^{\top} \Sigma^{-1} A_{k}+\tau\sum_{i:(i,t)\in\Omega}\boldsymbol{w}_{i}\boldsymbol{w}_{i}^\top+\Sigma^{-1}\right)^{-1}, \\
\boldsymbol{\mu}_{t}^{*}&=\Sigma_{t}^{*}\left(\sum_{k=1, h_{d}<t+h_{k} \leq T}^{d} A_{k}^{\top} \Sigma^{-1} \boldsymbol{\psi}_{t+h_{k}}+\tau\sum_{i:(i,t)\in\Omega}\boldsymbol{w}_{i}y_{it}+\Sigma^{-1}\sum_{k=1}^{d}A_{k}\boldsymbol{x}_{t-h_k}\right), \\
\end{aligned}
\end{equation}
where
$$\boldsymbol{\psi}_{t+h_k}=\boldsymbol{x}_{t+h_k}-\sum_{l=1,l\neq k}^{d}A_{l}\boldsymbol{x}_{t+h_k-h_l}.$$



In [36]:
def sample_factor_x(sparse_mat, binary_mat, time_lags, W, X, tau, A, Lambda_x):
    dim2, rank = X.shape
    d = time_lags.shape[0]
    mat0 = np.matmul(Lambda_x, A.T)
    mat1 = np.zeros((rank, rank, d))
    mat2 = np.zeros((rank, rank))
    for k in range(d):
        Ak = A[k * rank : (k + 1) * rank, :]
        mat1[:, :, k] = np.matmul(Ak, Lambda_x)
        mat2 += np.matmul(mat1[:, :, k], Ak.T)
    for t in range(dim2):
        pos0 = np.where(sparse_mat[:, t] != 0)
        Wt = W[pos0[0], :]
        Nt = np.zeros(rank)
        if t >= np.max(time_lags):
            Qt = np.matmul(mat0, X[t - time_lags, :].reshape([rank * d]))
            if t < dim2 - np.max(time_lags):
                Mt = mat2.copy()
                for k in range(d):
                    A0 = A.copy()
                    A0[k * rank : (k + 1) * rank, :] = 0
                    var5 = (X[t + time_lags[k], :] 
                            - np.matmul(A0.T, X[t + time_lags[k] 
                                                - time_lags, :].reshape([rank * d])))
                    Nt += np.matmul(mat1[:, :, k], var5)
            elif t >= dim2 - np.max(time_lags) and t < dim2 - np.min(time_lags):
                index = list(np.where(t + time_lags < dim2))[0]
                Mt = np.zeros((rank, rank))
                for k in index:
                    Ak = A[k * rank : (k + 1) * rank, :]
                    Mt += np.matmul(np.matmul(Ak, Lambda_x), Ak.T)
                    A0 = A.copy()
                    A0[k * rank : (k + 1) * rank, :] = 0
                    var5 = (X[t + time_lags[k], :] 
                            - np.matmul(A0.T, X[t + time_lags[k] 
                                                - time_lags, :].reshape([rank * d])))
                    Nt += np.matmul(np.matmul(Ak, Lambda_x), var5)
            inv_var_Lambda = inv(tau * np.matmul(Wt.T, Wt) + Mt + Lambda_x)
        elif t < np.max(time_lags):
            Qt = np.zeros(rank)
            index = list(np.where(t + time_lags >= np.max(time_lags)))[0]
            Mt = np.zeros((rank, rank))
            for k in index:
                Ak = A[k * rank : (k + 1) * rank, :]
                Mt += np.matmul(np.matmul(Ak, Lambda_x), Ak.T)
                A0 = A.copy()
                A0[k * rank : (k + 1) * rank, :] = 0
                var5 = (X[t + time_lags[k], :] 
                        - np.matmul(A0.T, X[t + time_lags[k] 
                                            - time_lags, :].reshape([rank * d])))
                Nt += np.matmul(np.matmul(Ak, Lambda_x), var5)
            inv_var_Lambda = inv(tau * np.matmul(Wt.T, Wt) + Mt + np.eye(rank))
        var_mu = tau * np.matmul(Wt.T, sparse_mat[pos0[0], t]) + Nt + Qt
        X[t, :] = mvnrnd(np.matmul(inv_var_Lambda, var_mu), inv_var_Lambda)
    return X

In [12]:
def sample_factor_x(sparse_mat, binary_mat, time_lags, W, X, tau, A, Lambda_x):
    dim2, rank = X.shape
    d = time_lags.shape[0]
    for t in range(dim2):
        pos0 = np.where(sparse_mat[:, t] != 0)
        Wt = W[pos0[0], :]
        Mt = np.zeros((rank, rank))
        Nt = np.zeros(rank)
        if t >= np.max(time_lags):
            Qt = np.matmul(Lambda_x, np.matmul(A.T, X[t - time_lags, :].reshape([rank * d])))
        if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
            index = list(range(0, d))
        elif t >= dim2 - np.max(time_lags) and t < dim2 - np.min(time_lags):
            index = list(np.where(t + time_lags < dim2))[0]
        elif t < np.max(time_lags):
            Qt = np.zeros(rank)
            index = list(np.where(t + time_lags >= np.max(time_lags)))[0]
        if t < dim2 - np.min(time_lags):
            for k in index:
                Ak = A[k * rank : (k + 1) * rank, :]
                Mt += np.matmul(np.matmul(Ak, Lambda_x), Ak.T)
                A0 = A.copy()
                A0[k * rank : (k + 1) * rank, :] = 0
                var5 = (X[t + time_lags[k], :] 
                        - np.matmul(A0.T, X[t + time_lags[k] - time_lags, :].reshape([rank * d])))
                Nt += np.matmul(np.matmul(Ak, Lambda_x), var5)
        var_mu = tau * np.matmul(Wt.T, sparse_mat[pos0[0], t]) + Nt + Qt
        if t < np.max(time_lags):
            inv_var_Lambda = inv(tau * np.matmul(Wt.T, Wt) + Mt + np.eye(rank))
        else:
            inv_var_Lambda = inv(tau * np.matmul(Wt.T, Wt) + Mt + Lambda_x)
        X[t, :] = mvnrnd(np.matmul(inv_var_Lambda, var_mu), inv_var_Lambda)
    return X

#### 4) Sampling Precision $\tau$

In [4]:
def sample_precision_tau(sparse_mat, mat_hat, position):
    var_alpha = 1e-6 + 0.5 * sparse_mat[position].shape[0]
    var_beta = 1e-6 + 0.5 * np.sum((sparse_mat - mat_hat)[position] ** 2)
    return np.random.gamma(var_alpha, 1 / var_beta)

#### 5) BTMF Implementation

- **Gibbs sampling**

    - Burn-in process
    - Sampling process


- **Imputation**


- **Prediction**

In [26]:
def BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter):
    """Bayesian Temporal Matrix Factorization, BTMF."""
    
    W = init["W"]
    X = init["X"]
    dim1, dim2 = sparse_mat.shape
    d = time_lags.shape[0]
    pos = np.where((dense_mat != 0) & (sparse_mat == 0))
    position = np.where(sparse_mat != 0)
    binary_mat = np.zeros((dim1, dim2))
    binary_mat[position] = 1
    tau = 1
    mat_hat_plus = np.zeros((dim1, dim2))
    for it in range(burn_iter + gibbs_iter):
        W = sample_factor_w(sparse_mat, binary_mat, W, X, tau)
        A, Sigma = sample_var_coefficient(X, time_lags)
        X = sample_factor_x(sparse_mat, binary_mat, time_lags, W, X, tau, A, inv(Sigma))
        mat_hat = np.matmul(W, X.T)
        tau = sample_precision_tau(sparse_mat, mat_hat, position)
        rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2) / dense_mat[pos].shape[0])
        if (it + 1) % 1 == 0 and it < burn_iter:
            print('Iteration: {}'.format(it + 1))
            print('RMSE: {:.6}'.format(rmse))
            print()
        if it + 1 > burn_iter:
            mat_hat_plus += mat_hat
    mat_hat = mat_hat_plus / gibbs_iter
    final_mape = np.sum(np.abs(dense_mat[pos] - mat_hat[pos]) / dense_mat[pos]) / dense_mat[pos].shape[0]
    final_rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2) / dense_mat[pos].shape[0])
    print('Imputation MAPE: {:.6}'.format(final_mape))
    print('Imputation RMSE: {:.6}'.format(final_rmse))
    print()
    return mat_hat

## 6 Spatiotemporal Missing Data Imputation


In [14]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
# =============================================================================

# binary_tensor = np.zeros(tensor.shape)
# for i1 in range(tensor.shape[0]):
#     for i2 in range(tensor.shape[1]):
#         binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
# binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])

sparse_mat = np.multiply(dense_mat, binary_mat)

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
burn_iter = 100
gibbs_iter = 10
BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 5.96841

Iteration: 2
RMSE: 5.79406

Iteration: 3
RMSE: 5.73579

Iteration: 4
RMSE: 5.37806

Iteration: 5
RMSE: 5.07442

Iteration: 6
RMSE: 4.84277

Iteration: 7
RMSE: 4.70153

Iteration: 8
RMSE: 4.59787

Iteration: 9
RMSE: 4.58369

Iteration: 10
RMSE: 4.57758

Iteration: 11
RMSE: 4.56305

Iteration: 12
RMSE: 4.50254

Iteration: 13
RMSE: 4.45239

Iteration: 14
RMSE: 4.43531

Iteration: 15
RMSE: 4.41852

Iteration: 16
RMSE: 4.39418

Iteration: 17
RMSE: 4.38041

Iteration: 18
RMSE: 4.37195

Iteration: 19
RMSE: 4.36142

Iteration: 20
RMSE: 4.3481

Iteration: 21
RMSE: 4.32629

Iteration: 22
RMSE: 4.31485

Iteration: 23
RMSE: 4.29651

Iteration: 24
RMSE: 4.28353

Iteration: 25
RMSE: 4.27995

Iteration: 26
RMSE: 4.27666

Iteration: 27
RMSE: 4.27007

Iteration: 28
RMSE: 4.26035

Iteration: 29
RMSE: 4.24952

Iteration: 30
RMSE: 4.24633

Iteration: 31
RMSE: 4.24226

Iteration: 32
RMSE: 4.23985

Iteration: 33
RMSE: 4.23764

Iteration: 34
RMSE: 4.2359

Iteration: 35
RMSE: 4.232

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
burn_iter = 1000
gibbs_iter = 100
BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 6.54248

Iteration: 2
RMSE: 5.76746

Iteration: 3
RMSE: 5.7331

Iteration: 4
RMSE: 5.74967

Iteration: 5
RMSE: 5.48408

Iteration: 6
RMSE: 5.36829

Iteration: 7
RMSE: 4.93222

Iteration: 8
RMSE: 4.70856

Iteration: 9
RMSE: 4.66498

Iteration: 10
RMSE: 4.74202

Iteration: 11
RMSE: 4.69944

Iteration: 12
RMSE: 4.64699

Iteration: 13
RMSE: 4.58376

Iteration: 14
RMSE: 4.56541

Iteration: 15
RMSE: 4.57934

Iteration: 16
RMSE: 4.53475

Iteration: 17
RMSE: 4.54741

Iteration: 18
RMSE: 4.60075

Iteration: 19
RMSE: 4.63094

Iteration: 20
RMSE: 4.59308

Iteration: 21
RMSE: 4.5828

Iteration: 22
RMSE: 4.54219

Iteration: 23
RMSE: 4.51025

Iteration: 24
RMSE: 4.482

Iteration: 25
RMSE: 4.46875

Iteration: 26
RMSE: 4.46078

Iteration: 27
RMSE: 4.45712

Iteration: 28
RMSE: 4.45436

Iteration: 29
RMSE: 4.4563

Iteration: 30
RMSE: 4.45248

Iteration: 31
RMSE: 4.45449

Iteration: 32
RMSE: 4.45683

Iteration: 33
RMSE: 4.4544

Iteration: 34
RMSE: 4.45835

Iteration: 35
RMSE: 4.46065



Iteration: 279
RMSE: 4.5352

Iteration: 280
RMSE: 4.53879

Iteration: 281
RMSE: 4.53062

Iteration: 282
RMSE: 4.53645

Iteration: 283
RMSE: 4.53852

Iteration: 284
RMSE: 4.53585

Iteration: 285
RMSE: 4.52965

Iteration: 286
RMSE: 4.5337

Iteration: 287
RMSE: 4.52432

Iteration: 288
RMSE: 4.5294

Iteration: 289
RMSE: 4.53376

Iteration: 290
RMSE: 4.52812

Iteration: 291
RMSE: 4.5261

Iteration: 292
RMSE: 4.5263

Iteration: 293
RMSE: 4.52704

Iteration: 294
RMSE: 4.52451

Iteration: 295
RMSE: 4.52665

Iteration: 296
RMSE: 4.52461

Iteration: 297
RMSE: 4.52631

Iteration: 298
RMSE: 4.51936

Iteration: 299
RMSE: 4.52083

Iteration: 300
RMSE: 4.51877

Iteration: 301
RMSE: 4.52033

Iteration: 302
RMSE: 4.52117

Iteration: 303
RMSE: 4.52341

Iteration: 304
RMSE: 4.52289

Iteration: 305
RMSE: 4.52642

Iteration: 306
RMSE: 4.52659

Iteration: 307
RMSE: 4.52489

Iteration: 308
RMSE: 4.51955

Iteration: 309
RMSE: 4.52345

Iteration: 310
RMSE: 4.52159

Iteration: 311
RMSE: 4.52437

Iteration: 312


Iteration: 554
RMSE: 4.53142

Iteration: 555
RMSE: 4.52967

Iteration: 556
RMSE: 4.52901

Iteration: 557
RMSE: 4.52616

Iteration: 558
RMSE: 4.5256

Iteration: 559
RMSE: 4.52306

Iteration: 560
RMSE: 4.52195

Iteration: 561
RMSE: 4.52115

Iteration: 562
RMSE: 4.53097

Iteration: 563
RMSE: 4.52992

Iteration: 564
RMSE: 4.52793

Iteration: 565
RMSE: 4.53062

Iteration: 566
RMSE: 4.52713

Iteration: 567
RMSE: 4.53011

Iteration: 568
RMSE: 4.52876

Iteration: 569
RMSE: 4.53496

Iteration: 570
RMSE: 4.53019

Iteration: 571
RMSE: 4.52878

Iteration: 572
RMSE: 4.5281

Iteration: 573
RMSE: 4.53244

Iteration: 574
RMSE: 4.52934

Iteration: 575
RMSE: 4.52825

Iteration: 576
RMSE: 4.53346

Iteration: 577
RMSE: 4.53242

Iteration: 578
RMSE: 4.53382

Iteration: 579
RMSE: 4.53286

Iteration: 580
RMSE: 4.53213

Iteration: 581
RMSE: 4.53003

Iteration: 582
RMSE: 4.52839

Iteration: 583
RMSE: 4.52916

Iteration: 584
RMSE: 4.53061

Iteration: 585
RMSE: 4.52901

Iteration: 586
RMSE: 4.53405

Iteration: 5

Iteration: 829
RMSE: 4.52942

Iteration: 830
RMSE: 4.53005

Iteration: 831
RMSE: 4.53429

Iteration: 832
RMSE: 4.53

Iteration: 833
RMSE: 4.52923

Iteration: 834
RMSE: 4.52564

Iteration: 835
RMSE: 4.52425

Iteration: 836
RMSE: 4.52404

Iteration: 837
RMSE: 4.52049

Iteration: 838
RMSE: 4.52268

Iteration: 839
RMSE: 4.52179

Iteration: 840
RMSE: 4.52469

Iteration: 841
RMSE: 4.52052

Iteration: 842
RMSE: 4.52798

Iteration: 843
RMSE: 4.52719

Iteration: 844
RMSE: 4.52822

Iteration: 845
RMSE: 4.52859

Iteration: 846
RMSE: 4.52124

Iteration: 847
RMSE: 4.52414

Iteration: 848
RMSE: 4.52514

Iteration: 849
RMSE: 4.52461

Iteration: 850
RMSE: 4.5199

Iteration: 851
RMSE: 4.51987

Iteration: 852
RMSE: 4.5177

Iteration: 853
RMSE: 4.52069

Iteration: 854
RMSE: 4.52308

Iteration: 855
RMSE: 4.52469

Iteration: 856
RMSE: 4.52429

Iteration: 857
RMSE: 4.52416

Iteration: 858
RMSE: 4.52467

Iteration: 859
RMSE: 4.52621

Iteration: 860
RMSE: 4.52895

Iteration: 861
RMSE: 4.52379

Iteration: 862


In [11]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
burn_iter = 1000
gibbs_iter = 100
BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 6.51248

Iteration: 2
RMSE: 5.76471

Iteration: 3
RMSE: 5.7968

Iteration: 4
RMSE: 6.01347

Iteration: 5
RMSE: 5.69031

Iteration: 6
RMSE: 5.16464

Iteration: 7
RMSE: 5.15277

Iteration: 8
RMSE: 4.92295

Iteration: 9
RMSE: 4.91495

Iteration: 10
RMSE: 4.77227

Iteration: 11
RMSE: 4.72594

Iteration: 12
RMSE: 4.61394

Iteration: 13
RMSE: 4.64089

Iteration: 14
RMSE: 4.6433

Iteration: 15
RMSE: 4.59453

Iteration: 16
RMSE: 4.59607

Iteration: 17
RMSE: 4.6745

Iteration: 18
RMSE: 4.7046

Iteration: 19
RMSE: 4.63844

Iteration: 20
RMSE: 4.57872

Iteration: 21
RMSE: 4.51034

Iteration: 22
RMSE: 4.47185

Iteration: 23
RMSE: 4.46469

Iteration: 24
RMSE: 4.46205

Iteration: 25
RMSE: 4.45455

Iteration: 26
RMSE: 4.46006

Iteration: 27
RMSE: 4.45566

Iteration: 28
RMSE: 4.45877

Iteration: 29
RMSE: 4.45974

Iteration: 30
RMSE: 4.46199

Iteration: 31
RMSE: 4.45311

Iteration: 32
RMSE: 4.45888

Iteration: 33
RMSE: 4.46109

Iteration: 34
RMSE: 4.46013

Iteration: 35
RMSE: 4.46354

Iteration: 279
RMSE: 4.51879

Iteration: 280
RMSE: 4.51481

Iteration: 281
RMSE: 4.51541

Iteration: 282
RMSE: 4.5148

Iteration: 283
RMSE: 4.5185

Iteration: 284
RMSE: 4.51774

Iteration: 285
RMSE: 4.52166

Iteration: 286
RMSE: 4.516

Iteration: 287
RMSE: 4.51967

Iteration: 288
RMSE: 4.51852

Iteration: 289
RMSE: 4.5162

Iteration: 290
RMSE: 4.51907

Iteration: 291
RMSE: 4.51571

Iteration: 292
RMSE: 4.51322

Iteration: 293
RMSE: 4.51498

Iteration: 294
RMSE: 4.51735

Iteration: 295
RMSE: 4.51235

Iteration: 296
RMSE: 4.51976

Iteration: 297
RMSE: 4.51846

Iteration: 298
RMSE: 4.52241

Iteration: 299
RMSE: 4.52177

Iteration: 300
RMSE: 4.5244

Iteration: 301
RMSE: 4.52854

Iteration: 302
RMSE: 4.5255

Iteration: 303
RMSE: 4.52634

Iteration: 304
RMSE: 4.52795

Iteration: 305
RMSE: 4.52338

Iteration: 306
RMSE: 4.52503

Iteration: 307
RMSE: 4.52705

Iteration: 308
RMSE: 4.5291

Iteration: 309
RMSE: 4.52955

Iteration: 310
RMSE: 4.52838

Iteration: 311
RMSE: 4.52821

Iteration: 312
RMS

Iteration: 554
RMSE: 4.52638

Iteration: 555
RMSE: 4.52198

Iteration: 556
RMSE: 4.52919

Iteration: 557
RMSE: 4.52802

Iteration: 558
RMSE: 4.5221

Iteration: 559
RMSE: 4.52323

Iteration: 560
RMSE: 4.52064

Iteration: 561
RMSE: 4.52077

Iteration: 562
RMSE: 4.52707

Iteration: 563
RMSE: 4.52291

Iteration: 564
RMSE: 4.52352

Iteration: 565
RMSE: 4.52184

Iteration: 566
RMSE: 4.52331

Iteration: 567
RMSE: 4.52276

Iteration: 568
RMSE: 4.52092

Iteration: 569
RMSE: 4.52569

Iteration: 570
RMSE: 4.51918

Iteration: 571
RMSE: 4.52004

Iteration: 572
RMSE: 4.5227

Iteration: 573
RMSE: 4.51989

Iteration: 574
RMSE: 4.5195

Iteration: 575
RMSE: 4.51989

Iteration: 576
RMSE: 4.52166

Iteration: 577
RMSE: 4.52223

Iteration: 578
RMSE: 4.52042

Iteration: 579
RMSE: 4.52357

Iteration: 580
RMSE: 4.51969

Iteration: 581
RMSE: 4.52226

Iteration: 582
RMSE: 4.52632

Iteration: 583
RMSE: 4.52401

Iteration: 584
RMSE: 4.51729

Iteration: 585
RMSE: 4.51871

Iteration: 586
RMSE: 4.52254

Iteration: 58

Iteration: 828
RMSE: 4.51798

Iteration: 829
RMSE: 4.52249

Iteration: 830
RMSE: 4.52131

Iteration: 831
RMSE: 4.52045

Iteration: 832
RMSE: 4.52463

Iteration: 833
RMSE: 4.52396

Iteration: 834
RMSE: 4.52775

Iteration: 835
RMSE: 4.52532

Iteration: 836
RMSE: 4.52213

Iteration: 837
RMSE: 4.52475

Iteration: 838
RMSE: 4.5238

Iteration: 839
RMSE: 4.52824

Iteration: 840
RMSE: 4.52861

Iteration: 841
RMSE: 4.52943

Iteration: 842
RMSE: 4.52382

Iteration: 843
RMSE: 4.52266

Iteration: 844
RMSE: 4.5228

Iteration: 845
RMSE: 4.51891

Iteration: 846
RMSE: 4.52124

Iteration: 847
RMSE: 4.5237

Iteration: 848
RMSE: 4.52419

Iteration: 849
RMSE: 4.51936

Iteration: 850
RMSE: 4.52145

Iteration: 851
RMSE: 4.51837

Iteration: 852
RMSE: 4.52385

Iteration: 853
RMSE: 4.52324

Iteration: 854
RMSE: 4.52268

Iteration: 855
RMSE: 4.5191

Iteration: 856
RMSE: 4.52725

Iteration: 857
RMSE: 4.52445

Iteration: 858
RMSE: 4.52211

Iteration: 859
RMSE: 4.52118

Iteration: 860
RMSE: 4.51885

Iteration: 861

In [7]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
burn_iter = 1000
gibbs_iter = 100
BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 6.47112

Iteration: 2
RMSE: 5.75809

Iteration: 3
RMSE: 5.76332

Iteration: 4
RMSE: 5.45379

Iteration: 5
RMSE: 5.29815

Iteration: 6
RMSE: 5.01902

Iteration: 7
RMSE: 5.21885

Iteration: 8
RMSE: 4.91487

Iteration: 9
RMSE: 4.79297

Iteration: 10
RMSE: 4.69447

Iteration: 11
RMSE: 4.76951

Iteration: 12
RMSE: 4.76809

Iteration: 13
RMSE: 4.63249

Iteration: 14
RMSE: 4.56245

Iteration: 15
RMSE: 4.53767

Iteration: 16
RMSE: 4.53919

Iteration: 17
RMSE: 4.53258

Iteration: 18
RMSE: 4.51677

Iteration: 19
RMSE: 4.51945

Iteration: 20
RMSE: 4.54172

Iteration: 21
RMSE: 4.54422

Iteration: 22
RMSE: 4.53043

Iteration: 23
RMSE: 4.50826

Iteration: 24
RMSE: 4.48526

Iteration: 25
RMSE: 4.47408

Iteration: 26
RMSE: 4.47039

Iteration: 27
RMSE: 4.46863

Iteration: 28
RMSE: 4.46985

Iteration: 29
RMSE: 4.47059

Iteration: 30
RMSE: 4.47263

Iteration: 31
RMSE: 4.47294

Iteration: 32
RMSE: 4.46789

Iteration: 33
RMSE: 4.46396

Iteration: 34
RMSE: 4.46976

Iteration: 35
RMSE: 4.4

Iteration: 279
RMSE: 4.52628

Iteration: 280
RMSE: 4.52704

Iteration: 281
RMSE: 4.52969

Iteration: 282
RMSE: 4.52869

Iteration: 283
RMSE: 4.52985

Iteration: 284
RMSE: 4.52869

Iteration: 285
RMSE: 4.52986

Iteration: 286
RMSE: 4.53465

Iteration: 287
RMSE: 4.52804

Iteration: 288
RMSE: 4.52938

Iteration: 289
RMSE: 4.52609

Iteration: 290
RMSE: 4.52868

Iteration: 291
RMSE: 4.52671

Iteration: 292
RMSE: 4.52586

Iteration: 293
RMSE: 4.51926

Iteration: 294
RMSE: 4.52362

Iteration: 295
RMSE: 4.52561

Iteration: 296
RMSE: 4.52389

Iteration: 297
RMSE: 4.52144

Iteration: 298
RMSE: 4.52186

Iteration: 299
RMSE: 4.52136

Iteration: 300
RMSE: 4.52664

Iteration: 301
RMSE: 4.52172

Iteration: 302
RMSE: 4.52486

Iteration: 303
RMSE: 4.52058

Iteration: 304
RMSE: 4.52033

Iteration: 305
RMSE: 4.52002

Iteration: 306
RMSE: 4.51633

Iteration: 307
RMSE: 4.5173

Iteration: 308
RMSE: 4.5241

Iteration: 309
RMSE: 4.52262

Iteration: 310
RMSE: 4.52145

Iteration: 311
RMSE: 4.52045

Iteration: 3

Iteration: 553
RMSE: 4.52077

Iteration: 554
RMSE: 4.52313

Iteration: 555
RMSE: 4.52136

Iteration: 556
RMSE: 4.52352

Iteration: 557
RMSE: 4.52722

Iteration: 558
RMSE: 4.5264

Iteration: 559
RMSE: 4.52613

Iteration: 560
RMSE: 4.52595

Iteration: 561
RMSE: 4.52221

Iteration: 562
RMSE: 4.52655

Iteration: 563
RMSE: 4.52799

Iteration: 564
RMSE: 4.53185

Iteration: 565
RMSE: 4.524

Iteration: 566
RMSE: 4.52454

Iteration: 567
RMSE: 4.52319

Iteration: 568
RMSE: 4.52439

Iteration: 569
RMSE: 4.52221

Iteration: 570
RMSE: 4.52588

Iteration: 571
RMSE: 4.52167

Iteration: 572
RMSE: 4.52094

Iteration: 573
RMSE: 4.52435

Iteration: 574
RMSE: 4.52241

Iteration: 575
RMSE: 4.52247

Iteration: 576
RMSE: 4.52121

Iteration: 577
RMSE: 4.52344

Iteration: 578
RMSE: 4.52262

Iteration: 579
RMSE: 4.52264

Iteration: 580
RMSE: 4.52421

Iteration: 581
RMSE: 4.52649

Iteration: 582
RMSE: 4.52475

Iteration: 583
RMSE: 4.52607

Iteration: 584
RMSE: 4.52561

Iteration: 585
RMSE: 4.5274

Iteration: 586

Iteration: 828
RMSE: 4.53102

Iteration: 829
RMSE: 4.53026

Iteration: 830
RMSE: 4.52896

Iteration: 831
RMSE: 4.53003

Iteration: 832
RMSE: 4.53628

Iteration: 833
RMSE: 4.53137

Iteration: 834
RMSE: 4.53044

Iteration: 835
RMSE: 4.53618

Iteration: 836
RMSE: 4.53267

Iteration: 837
RMSE: 4.53497

Iteration: 838
RMSE: 4.5334

Iteration: 839
RMSE: 4.53509

Iteration: 840
RMSE: 4.5316

Iteration: 841
RMSE: 4.53321

Iteration: 842
RMSE: 4.53299

Iteration: 843
RMSE: 4.5319

Iteration: 844
RMSE: 4.53203

Iteration: 845
RMSE: 4.52704

Iteration: 846
RMSE: 4.52248

Iteration: 847
RMSE: 4.52411

Iteration: 848
RMSE: 4.52254

Iteration: 849
RMSE: 4.52472

Iteration: 850
RMSE: 4.52127

Iteration: 851
RMSE: 4.52597

Iteration: 852
RMSE: 4.52544

Iteration: 853
RMSE: 4.52188

Iteration: 854
RMSE: 4.52123

Iteration: 855
RMSE: 4.52443

Iteration: 856
RMSE: 4.52403

Iteration: 857
RMSE: 4.52554

Iteration: 858
RMSE: 4.52226

Iteration: 859
RMSE: 4.52629

Iteration: 860
RMSE: 4.5234

Iteration: 861

In [24]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
burn_iter = 1000
gibbs_iter = 100
BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 6.58118

Iteration: 2
RMSE: 5.77303

Iteration: 3
RMSE: 5.78238

Iteration: 4
RMSE: 5.56501

Iteration: 5
RMSE: 5.38285

Iteration: 6
RMSE: 5.71446

Iteration: 7
RMSE: 5.20444

Iteration: 8
RMSE: 4.88352

Iteration: 9
RMSE: 4.81263

Iteration: 10
RMSE: 4.81308

Iteration: 11
RMSE: 4.7444

Iteration: 12
RMSE: 4.68449

Iteration: 13
RMSE: 4.6307

Iteration: 14
RMSE: 4.56552

Iteration: 15
RMSE: 4.55113

Iteration: 16
RMSE: 4.59715

Iteration: 17
RMSE: 4.64085

Iteration: 18
RMSE: 4.59975

Iteration: 19
RMSE: 4.56311

Iteration: 20
RMSE: 4.54207

Iteration: 21
RMSE: 4.52086

Iteration: 22
RMSE: 4.48656

Iteration: 23
RMSE: 4.47396

Iteration: 24
RMSE: 4.46452

Iteration: 25
RMSE: 4.46573

Iteration: 26
RMSE: 4.46231

Iteration: 27
RMSE: 4.46396

Iteration: 28
RMSE: 4.45965

Iteration: 29
RMSE: 4.45974

Iteration: 30
RMSE: 4.45499

Iteration: 31
RMSE: 4.46041

Iteration: 32
RMSE: 4.46123

Iteration: 33
RMSE: 4.4613

Iteration: 34
RMSE: 4.46542

Iteration: 35
RMSE: 4.4667

Iteration: 279
RMSE: 4.51034

Iteration: 280
RMSE: 4.5149

Iteration: 281
RMSE: 4.5151

Iteration: 282
RMSE: 4.51799

Iteration: 283
RMSE: 4.5149

Iteration: 284
RMSE: 4.51351

Iteration: 285
RMSE: 4.5216

Iteration: 286
RMSE: 4.51897

Iteration: 287
RMSE: 4.51443

Iteration: 288
RMSE: 4.51846

Iteration: 289
RMSE: 4.51301

Iteration: 290
RMSE: 4.5174

Iteration: 291
RMSE: 4.52079

Iteration: 292
RMSE: 4.52457

Iteration: 293
RMSE: 4.52765

Iteration: 294
RMSE: 4.52491

Iteration: 295
RMSE: 4.52534

Iteration: 296
RMSE: 4.52242

Iteration: 297
RMSE: 4.52022

Iteration: 298
RMSE: 4.52292

Iteration: 299
RMSE: 4.51863

Iteration: 300
RMSE: 4.51722

Iteration: 301
RMSE: 4.52388

Iteration: 302
RMSE: 4.52293

Iteration: 303
RMSE: 4.52192

Iteration: 304
RMSE: 4.51701

Iteration: 305
RMSE: 4.51371

Iteration: 306
RMSE: 4.51491

Iteration: 307
RMSE: 4.51734

Iteration: 308
RMSE: 4.51443

Iteration: 309
RMSE: 4.51835

Iteration: 310
RMSE: 4.51529

Iteration: 311
RMSE: 4.51424

Iteration: 312


Iteration: 554
RMSE: 4.52312

Iteration: 555
RMSE: 4.51669

Iteration: 556
RMSE: 4.51714

Iteration: 557
RMSE: 4.51614

Iteration: 558
RMSE: 4.52285

Iteration: 559
RMSE: 4.52229

Iteration: 560
RMSE: 4.51909

Iteration: 561
RMSE: 4.52079

Iteration: 562
RMSE: 4.52504

Iteration: 563
RMSE: 4.52719

Iteration: 564
RMSE: 4.52445

Iteration: 565
RMSE: 4.52747

Iteration: 566
RMSE: 4.5224

Iteration: 567
RMSE: 4.5251

Iteration: 568
RMSE: 4.52302

Iteration: 569
RMSE: 4.52702

Iteration: 570
RMSE: 4.53169

Iteration: 571
RMSE: 4.52663

Iteration: 572
RMSE: 4.528

Iteration: 573
RMSE: 4.52764

Iteration: 574
RMSE: 4.53165

Iteration: 575
RMSE: 4.53384

Iteration: 576
RMSE: 4.53353

Iteration: 577
RMSE: 4.53533

Iteration: 578
RMSE: 4.53886

Iteration: 579
RMSE: 4.53261

Iteration: 580
RMSE: 4.53036

Iteration: 581
RMSE: 4.52705

Iteration: 582
RMSE: 4.52651

Iteration: 583
RMSE: 4.52788

Iteration: 584
RMSE: 4.52737

Iteration: 585
RMSE: 4.5278

Iteration: 586
RMSE: 4.52587

Iteration: 587


Iteration: 829
RMSE: 4.5205

Iteration: 830
RMSE: 4.51996

Iteration: 831
RMSE: 4.51882

Iteration: 832
RMSE: 4.52028

Iteration: 833
RMSE: 4.51737

Iteration: 834
RMSE: 4.52161

Iteration: 835
RMSE: 4.51933

Iteration: 836
RMSE: 4.5197

Iteration: 837
RMSE: 4.51331

Iteration: 838
RMSE: 4.51213

Iteration: 839
RMSE: 4.50996

Iteration: 840
RMSE: 4.51457

Iteration: 841
RMSE: 4.51505

Iteration: 842
RMSE: 4.51632

Iteration: 843
RMSE: 4.51488

Iteration: 844
RMSE: 4.51638

Iteration: 845
RMSE: 4.51624

Iteration: 846
RMSE: 4.51723

Iteration: 847
RMSE: 4.51612

Iteration: 848
RMSE: 4.51197

Iteration: 849
RMSE: 4.51654

Iteration: 850
RMSE: 4.51542

Iteration: 851
RMSE: 4.51654

Iteration: 852
RMSE: 4.51468

Iteration: 853
RMSE: 4.51466

Iteration: 854
RMSE: 4.51378

Iteration: 855
RMSE: 4.51421

Iteration: 856
RMSE: 4.51825

Iteration: 857
RMSE: 4.51726

Iteration: 858
RMSE: 4.51652

Iteration: 859
RMSE: 4.51344

Iteration: 860
RMSE: 4.51656

Iteration: 861
RMSE: 4.51556

Iteration: 8

In [13]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values

missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [14]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 50
time_lags = np.array([1, 2, 288])
init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 1100
maxiter2 = 100
BTMF(dense_mat, sparse_mat, init, rank, time_lags, maxiter1, maxiter2)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 8.54652

Iteration: 2
RMSE: 8.25793

Iteration: 3
RMSE: 6.82373

Iteration: 4
RMSE: 6.29088

Iteration: 5
RMSE: 6.26116

Iteration: 6
RMSE: 6.11663

Iteration: 7
RMSE: 5.9606

Iteration: 8
RMSE: 5.94366

Iteration: 9
RMSE: 5.90364

Iteration: 10
RMSE: 5.79692

Iteration: 11
RMSE: 5.77708

Iteration: 12
RMSE: 5.75393

Iteration: 13
RMSE: 5.64862

Iteration: 14
RMSE: 5.54195

Iteration: 15
RMSE: 5.47609

Iteration: 16
RMSE: 5.42693

Iteration: 17
RMSE: 5.42135

Iteration: 18
RMSE: 5.41639

Iteration: 19
RMSE: 5.4097

Iteration: 20
RMSE: 5.39612

Iteration: 21
RMSE: 5.3333

Iteration: 22
RMSE: 5.23819

Iteration: 23
RMSE: 5.1996

Iteration: 24
RMSE: 5.18862

Iteration: 25
RMSE: 5.1844

Iteration: 26
RMSE: 5.17956

Iteration: 27
RMSE: 5.17783

Iteration: 28
RMSE: 5.17189

Iteration: 29
RMSE: 5.15882

Iteration: 30
RMSE: 5.13258

Iteration: 31
RMSE: 5.1249

Iteration: 32
RMSE: 5.1116

Iteration: 33
RMSE: 5.07737

Iteration: 34
RMSE: 5.05637

Iteration: 35
RMSE: 5.04501

I

Iteration: 279
RMSE: 4.0684

Iteration: 280
RMSE: 4.06494

Iteration: 281
RMSE: 4.06389

Iteration: 282
RMSE: 4.06309

Iteration: 283
RMSE: 4.06212

Iteration: 284
RMSE: 4.06207

Iteration: 285
RMSE: 4.05934

Iteration: 286
RMSE: 4.0612

Iteration: 287
RMSE: 4.05956

Iteration: 288
RMSE: 4.05898

Iteration: 289
RMSE: 4.06115

Iteration: 290
RMSE: 4.06128

Iteration: 291
RMSE: 4.0611

Iteration: 292
RMSE: 4.0583

Iteration: 293
RMSE: 4.06172

Iteration: 294
RMSE: 4.06312

Iteration: 295
RMSE: 4.06149

Iteration: 296
RMSE: 4.06089

Iteration: 297
RMSE: 4.0614

Iteration: 298
RMSE: 4.06076

Iteration: 299
RMSE: 4.05725

Iteration: 300
RMSE: 4.05363

Iteration: 301
RMSE: 4.05756

Iteration: 302
RMSE: 4.05997

Iteration: 303
RMSE: 4.05964

Iteration: 304
RMSE: 4.05655

Iteration: 305
RMSE: 4.05586

Iteration: 306
RMSE: 4.05818

Iteration: 307
RMSE: 4.05871

Iteration: 308
RMSE: 4.05555

Iteration: 309
RMSE: 4.05488

Iteration: 310
RMSE: 4.05449

Iteration: 311
RMSE: 4.05731

Iteration: 312


/Users/xinyuchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: covariance is not positive-semidefinite.


Iteration: 394
RMSE: 4.04085

Iteration: 395
RMSE: 4.04268

Iteration: 396
RMSE: 4.0425

Iteration: 397
RMSE: 4.04042

Iteration: 398
RMSE: 4.04092

Iteration: 399
RMSE: 4.03779

Iteration: 400
RMSE: 4.04163

Iteration: 401
RMSE: 4.04327

Iteration: 402
RMSE: 4.0422

Iteration: 403
RMSE: 4.04504

Iteration: 404
RMSE: 4.04549

Iteration: 405
RMSE: 4.04062

Iteration: 406
RMSE: 4.03777

Iteration: 407
RMSE: 4.04032

Iteration: 408
RMSE: 4.03954

Iteration: 409
RMSE: 4.04377

Iteration: 410
RMSE: 4.04143

Iteration: 411
RMSE: 4.03972

Iteration: 412
RMSE: 4.04493

Iteration: 413
RMSE: 4.04041

Iteration: 414
RMSE: 4.03843

Iteration: 415
RMSE: 4.03943

Iteration: 416
RMSE: 4.04237

Iteration: 417
RMSE: 4.03858

Iteration: 418
RMSE: 4.03928

Iteration: 419
RMSE: 4.04381

Iteration: 420
RMSE: 4.04177

Iteration: 421
RMSE: 4.03933

Iteration: 422
RMSE: 4.04199

Iteration: 423
RMSE: 4.03973

Iteration: 424
RMSE: 4.04423

Iteration: 425
RMSE: 4.03968

Iteration: 426
RMSE: 4.04267

Iteration: 4

LinAlgError: Matrix is not positive definite

## 7 Multivariate Time Series Prediction


In [39]:
def BTMF_burn(dense_mat, sparse_mat, init, time_lags, burn_iter):
    
    W = init["W"]
    X = init["X"]
    dim1, dim2 = sparse_mat.shape
    d = time_lags.shape[0]
    pos = np.where((dense_mat != 0) & (sparse_mat == 0))
    position = np.where(sparse_mat != 0)
    binary_mat = np.zeros((dim1, dim2))
    binary_mat[position] = 1
    tau = 1
    for it in range(burn_iter):
        W = sample_factor_w(sparse_mat, binary_mat, W, X, tau)
        A, Sigma = sample_var_coefficient(X, time_lags)
        X = sample_factor_x(sparse_mat, binary_mat, time_lags, W, X, tau, A, inv(Sigma))
        mat_hat = np.matmul(W, X.T)
        tau = sample_precision_tau(sparse_mat, mat_hat, position)
        rmse = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos]) ** 2) / dense_mat[pos].shape[0])
        if (it + 1) % 1 == 0 and it < burn_iter:
            print('Iteration: {}'.format(it + 1))
            print('RMSE: {:.6}'.format(rmse))
            print()
    return W, X, tau, A

In [49]:
def BTMF_4cast(mat, binary_mat, num_step, time_lags, init, gibbs_iter):
    """Forecast (`4cast`) time series with Bayesian Temporal Matrix Factorization (BTMF)."""
    
    W = init["W"]
    X = init["X"]
    tau = init["tau"]
    A = init["A"]
    rank = W.shape[1]
    d = time_lags.shape[0]
    mat_hat = np.zeros((W.shape[0], num_step, gibbs_iter))
    for it in range(gibbs_iter):
        W = sample_factor_w(mat, binary_mat, W, X, tau)
        A, Sigma = sample_var_coefficient(X, time_lags)
        X = sample_factor_x(sparse_mat, binary_mat, time_lags, W, X, tau, A, inv(Sigma))
        X_new = X.copy()
        for t in range(num_step):
            var = X_new[X.shape[0] + t - 1 - time_lags, :].reshape([rank * d])
            X_new = np.append(X_new, np.matmul(A.T, var).reshape([1, rank]), axis = 0)
        mat_hat[:, :, it] = np.random.normal(np.matmul(W, X_new[-1 - num_step : -1, :].T), 1 / tau) # dim1 * num_step
    return mat_hat, W, X_new, tau, A

In [53]:
def forecastor(dense_mat, sparse_mat, init, time_lags,
               num_roll, start_time, num_step, burn_iter, gibbs_iter):
    
    W, X, tau, A = BTMF_burn(dense_mat[:, : start_time], sparse_mat[:, : start_time],
                             init, time_lags, burn_iter)
    result = np.zeros((W.shape[0], num_roll * num_step, gibbs_iter))
    for t in range(num_roll):
        mat = sparse_mat[:, : start_time + t * num_step]
        print(mat.shape[1])
        position = np.where(mat != 0)
        binary_mat = mat.copy()
        binary_mat[position] = 1
        init = {"W": W, "X": X, "tau": tau, "A": A}
        mat_hat, W, X, tau, A = BTMF_4cast(mat, binary_mat, 
                                           num_step, time_lags, init, gibbs_iter)
        result[:, t * num_step : (t + 1) * num_step, :] = mat_hat
        
    mat_hat0 = np.mean(result, axis = 2)
    small_dense_mat = dense_mat[:, start_time : dense_mat.shape[1]]
    pos = np.where(small_dense_mat != 0)
    final_mape = np.sum(np.abs(small_dense_mat[pos] - 
                               mat_hat0[pos]) / small_dense_mat[pos]) / small_dense_mat[pos].shape[0]
    final_rmse = np.sqrt(np.sum((small_dense_mat[pos] - 
                                 mat_hat0[pos]) ** 2) / small_dense_mat[pos].shape[0])
    print('Final MAPE: {:.6}'.format(final_mape))
    print('Final RMSE: {:.6}'.format(final_rmse))
    print()
    return result

In [31]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario
### Set the RM scenario by:
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
# =============================================================================

# binary_tensor = np.zeros(tensor.shape)
# for i1 in range(tensor.shape[0]):
#     for i2 in range(tensor.shape[1]):
#         binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
# binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])

sparse_mat = np.multiply(dense_mat, binary_mat)

In [54]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
num_step = 6
num_roll = int(144 * 5 / num_step)
start_time = dim2 - num_roll * num_step
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(start_time, rank)}
burn_iter = 10
gibbs_iter = 2
result = forecastor(dense_mat, sparse_mat, init, time_lags,
                    num_roll, start_time, num_step, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 5.94597

Iteration: 2
RMSE: 5.74879

Iteration: 3
RMSE: 5.70634

Iteration: 4
RMSE: 5.47132

Iteration: 5
RMSE: 4.99418

Iteration: 6
RMSE: 4.76256

Iteration: 7
RMSE: 4.68645

Iteration: 8
RMSE: 4.5744

Iteration: 9
RMSE: 4.55261

Iteration: 10
RMSE: 4.54463

8064
8070
8076
8082
8088
8094
8100
8106
8112
8118
8124
8130
8136
8142
8148
8154
8160
8166
8172
8178
8184
8190
8196
8202
8208
8214
8220
8226
8232
8238
8244
8250
8256
8262
8268
8274
8280
8286
8292
8298
8304
8310
8316
8322
8328
8334
8340
8346
8352
8358
8364
8370
8376
8382
8388
8394
8400
8406
8412
8418
8424
8430
8436
8442
8448
8454
8460
8466
8472
8478
8484
8490
8496
8502
8508
8514
8520
8526
8532
8538
8544
8550
8556
8562
8568
8574
8580
8586
8592
8598
8604
8610
8616
8622
8628
8634
8640
8646
8652
8658
8664
8670
8676
8682
8688
8694
8700
8706
8712
8718
8724
8730
8736
8742
8748
8754
8760
8766
8772
8778
Final MAPE: 0.372138
Final RMSE: 15.0209

Running time: 762 seconds


In [ ]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
time_lags = np.array([1, 2, 144])
num_step = 6
num_roll = int(144 * 5 / num_step)
start_time = dim2 - num_roll * num_step
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(start_time, rank)}
burn_iter = 100
gibbs_iter = 20
result = forecastor(dense_mat, sparse_mat, init, time_lags,
                    num_roll, start_time, num_step, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 1
RMSE: 5.94449

Iteration: 2
RMSE: 5.75295

Iteration: 3
RMSE: 5.71641

Iteration: 4
RMSE: 5.53581

Iteration: 5
RMSE: 5.0634

Iteration: 6
RMSE: 4.88969

Iteration: 7
RMSE: 4.74457

Iteration: 8
RMSE: 4.68477

Iteration: 9
RMSE: 4.56635

Iteration: 10
RMSE: 4.54667

Iteration: 11
RMSE: 4.53713

Iteration: 12
RMSE: 4.50478

Iteration: 13
RMSE: 4.44558

Iteration: 14
RMSE: 4.41896

Iteration: 15
RMSE: 4.40447

Iteration: 16
RMSE: 4.39169

Iteration: 17
RMSE: 4.3801

Iteration: 18
RMSE: 4.36146

Iteration: 19
RMSE: 4.34012

Iteration: 20
RMSE: 4.323

Iteration: 21
RMSE: 4.2991

Iteration: 22
RMSE: 4.27874

Iteration: 23
RMSE: 4.26798

Iteration: 24
RMSE: 4.26194

Iteration: 25
RMSE: 4.25591

Iteration: 26
RMSE: 4.25072

Iteration: 27
RMSE: 4.24422

Iteration: 28
RMSE: 4.24307

Iteration: 29
RMSE: 4.23493

Iteration: 30
RMSE: 4.22458

Iteration: 31
RMSE: 4.21628

Iteration: 32
RMSE: 4.21207

Iteration: 33
RMSE: 4.21048

Iteration: 34
RMSE: 4.20769

Iteration: 35
RMSE: 4.20458
